In [2]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers import pipeline
import torch

model_id="google/gemma-3-1b-it"
# model_id="Qwen/Qwen3Guard-Gen-4B"
device="cpu"

tokenizer = AutoTokenizer.from_pretrained(model_id)

model = AutoModelForCausalLM.from_pretrained(model_id, dtype="auto", device_map=device)

# Try restarting your kernel if having issues


In [28]:
generation_pipeline = pipeline(task="text-generation", model=model, tokenizer=tokenizer)

generation_pipeline("What is your name?", max_new_tokens=40)

Device set to use cpu


[{'generated_text': 'What is your name?\n\n(Pause for response)\n\nOkay, thank you. I’m Alex.\n\nHow are you doing today?\n\n(Pause for response)\n\nI’m doing well, thanks for asking'}]

In [ ]:
input_prompt = [
    "What is 1 + 1?",
    "The capital of the US is",
    "gibberish"
]

tokenized = tokenizer(input_prompt, return_tensors="pt", padding=True).to(device)

print(tokenized["input_ids"])

tensor([[     2,   3689,    563, 236743, 236770,    900, 236743, 236770, 236881],
        [     0,      0,      2,    818,   5279,    529,    506,   2590,    563],
        [     0,      0,      0,      0,      0,      2, 148673,    793,   1044]])


In [37]:
prompt_template = [
    {
        "role": "system",
        "content": "You are a helpful AI assitant that helps engineers and real estate developers"
    },
    {
        "role": "user",
        "content": "Where does the sun rise?"
    },
    {
        "role": "assistant",
        "content": "A developer is"
    }
]

tokenizer.pad_token = tokenizer.eos_token

tokenized = tokenizer.apply_chat_template(
    prompt_template,
    add_generation_prompt=True,
    tokenize=False,
    padding=True,
    return_tensors="pt"
)

print(tokenized)
# Note, google/gemma-3-1b-it does not use system roles. The system prompt is injected directly into the first user message

<bos><start_of_turn>user
You are a helpful AI assitant that helps engineers and real estate developers

Where does the sun rise?<end_of_turn>
<start_of_turn>model
A developer is<end_of_turn>
<start_of_turn>model



In [ ]:
prompt_template = [
    {
        "role": "system",
        "content": "You are a helpful AI assitant that helps engineers and real estate developers"
    },
    {
        "role": "user",
        "content": "Calculate the design occupant load for a 500 sq ft Library Reading Room in a new university building in Manhattan, per NYC 2022 Building Code."
    },
    {
        "role": "assistant",
        "content": "Aye Aye"
    }
]

tokenizer.pad_token = tokenizer.eos_token

tokenized = tokenizer.apply_chat_template(
    prompt_template,
    add_generation_prompt=False,
    continue_final_message=True,
    tokenize=True,
    padding=True,
    return_tensors="pt"
)

print(tokenized)
# Note, google/gemma-3-1b-it does not use system roles. The system prompt is injected directly into the first user message

tensor([[     2,    105,   2364,    107,   3048,    659,    496,  11045,  12498,
           1188,  42563,    600,   7351,  22072,    532,   1759,   9350,  18270,
            108,  10936,   1677,    506,   3768,   8570, 236881,    106,    107,
            105,   4368,    107, 215784, 140871]])


In [54]:
out = model.generate(tokenized.to(device), max_new_tokens=20)
decoded = tokenizer.batch_decode(out)
print(decoded[0])

<bos><start_of_turn>user
You are a helpful AI assitant that helps engineers and real estate developers

Where does the sun rise?<end_of_turn>
<start_of_turn>model
Aye Aye, Captain! That's a fantastic question, and one that's been pondered by sailors and


In [39]:
tokenizer.batch_decode(tokenized["input_ids"])

TypeError: string indices must be integers, not 'str'

In [34]:
tokenized["attention_mask"]

C:\Users\borow\AppData\Local\Temp\ipykernel_31904\2594409403.py:1: UserWarning: Using a non-tuple sequence for multidimensional indexing is deprecated and will be changed in pytorch 2.9; use x[tuple(seq)] instead of x[seq]. In pytorch 2.9 this will be interpreted as tensor index, x[torch.tensor(seq)], which will result either in an error or a different result (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\pytorch\torch\csrc\autograd\python_variable_indexing.cpp:351.)
  tokenized["attention_mask"]


IndexError: too many indices for tensor of dimension 2